# Prevendo preços de carros


### Neste módulo iremos usar alguns modelos de regressão para prever preços de carros. Além disso iremos calcular métricas para cada um e no final escolher o que melhor prediz os valores evitando o overfitting.

Antes de tudo faremos os imports das bibliotecas que iremos usar por agora

In [20]:
# 
import pandas as pd
import matplotlib.pyplot as plt

#
dataframe = pd.read_csv(r'D:\Documentos\UFCG\IA\ProjetoPredicao\predicao-precos\true_car_listings.csv')
df_counts = dataframe[["Make"]].groupby("Make").size().reset_index(name='Counts')
df_counts = df_counts.sort_values(by='Counts', ascending=False)

Para explicar melhor o passo anterior, nós carregamos o dataset e criamos um novo objeto de dataframe chamado df_counts com o objetivo de plotar um gráfico para saber as fabricantes e o número de veículos que cada uma tem no dataset. O próximo bloco de código irá nos mostrar isso.

In [ ]:
#
plt.xlabel('FABRICANTE')
plt.ylabel('NUMERO DE VEICULOS')
plt.xticks(rotation=90) #Rotaciona os nomes do eixo X em 90 graus
plt.bar(df_counts['Make'], df_counts['Counts'])

<BarContainer object of 58 artists>